In [7]:
from pathlib import Path
from IPython.display import HTML, display
import ipywidgets as widgets
import datetime
import pandas as pd
import numpy as np
#print(Path.cwd().parent.parent.parent)
base_path = Path.cwd().parent.parent.parent / "runs"
#img_path = Path.cwd() / "img.jpg"
# display(Image(filename=img_path))
p = Path(base_path)
#p.as_uri()

In [8]:
tsts = ["k8s_taewa_3lite_","asg_taewa_3lite_", "k8s_apache_3_", "k8s_taewa_3_", "asg_apache_3_", "asg_taewa_3_"]
tsts = tsts + ["asg_taewa_3extra_", "k8s_taewa_3extra_", "asg_raupi_3_", "k8s_raupi_3_","k8s_taewa_6_", "asg_taewa_6_"]
tsts = tsts + ["asg_riwai_3_", "k8s_riwai_3_"]
test = widgets.Dropdown(
       options=tsts,
       value='k8s_apache_3_',
       description='Test:')
all = widgets.Checkbox(value=True,
       description='All')
graphs=["QPS_estimatedProcessedBytes.png","QPS_cpuUtilization.png","QPS_groupInServiceCapacity.png"]
date_filter = widgets.Text(
       value='2022.0[3-9]', # use glob pattern http://pymotw.com/2/glob/
       description='Filter', )
box = widgets.HBox([date_filter, all, test ])
display(box)

In [17]:
#print(test._options_values)
tests=[]
if all.value:
    tests=test._options_values;
else:
    tests.append(test.value)

for tst in tests:
    print('\n','='*40,f'\nprocessing {tst}\n')
    search=f'*{date_filter.value}*{tst}*'
    nw = datetime.datetime.now()
    count=0
    iList=[]
    #avg_qps_list=[]
    #avg_cpu_list=[]
    #avg_duration_list=[]
    total_err_list=[]

    all_qps_list=[]
    all_cpu_list=[]
    all_duration_list=[]
    all_err_list=[]

    for i in p.glob(search):
        print(i)
        count+=1
        dir = i.name
        platform=dir.split("_")[2]
        if not (i/"csv").exists():
            line = f"<tr><td rowspan='1'><b>No data found, skip: {dir}</b></td></tr>"
            iList.append(line)
            continue
        ##================ get json Fortio data ================
        #print(i)
        json_paths = (i/"csv").glob('*.json')
        fortio_path=list(json_paths)[0]

        df = pd.read_json(fortio_path, lines=True)
        df['StartTime'] = pd.to_datetime(df['StartTime'])
        df.set_index(['StartTime'],inplace=True)

        #qps= df[["ActualQPS","NumThreads"]]
        #mydf = df[df['Labels'] == 'k8s_apache_3-performance-1']

        # get performance QPS
        perf_df = df[df['Labels'].str.contains('performance-[1-3]', regex=True)]
        #perf_data=perf_df.ActualQPS
        #qps_lst=list(map(int,perf_data))
        qps_lst=list(perf_df.ActualQPS)
        avg_qps=np.mean(qps_lst)
        all_qps_list.extend(qps_lst)
        #avg_qps_list.append(avg_qps)
        
        # print("max all cpu:",max(all_cpu_list))
        # print("all_cpu_list:",all_cpu_list)

        # get scaling start times
        scaling_starts_df = df[df['Labels'].str.contains('scaling-.-1$', regex=True)]
        #print(df)
        #print(scaling_df['Labels'])

        aws_metric0="backendConnectionErrors"
        c_path_err=list((i/"csv").glob('*'+aws_metric0+'.csv'))[0]
        err_df = pd.read_csv(c_path_err, parse_dates=['datetime'], index_col="datetime")

        ## ================ PERFORMANCE CloudWatch data ================
        aws_metric1="cpuUtilization"
        csv_paths = (i/"csv").glob('*'+aws_metric1+'.csv')
        c_path=list(csv_paths)[0]
        mdf = pd.read_csv(c_path, parse_dates=['datetime'], index_col="datetime")
        avg_perfomance_cpu_list=[]
        #print(c_path)
        for index,value in perf_df.iterrows():
            perf_start=index#pd.to_datetime(stime)
            #print(perf_start)
            
            # get 4 max:
            # mmdf=mdf[perf_start:perf_start+ pd.Timedelta(5, "m")]
            # mmmdf=mmdf.nlargest(4, aws_metric1).sort_index()
            # get 3 mid:
            
            mmmdf=mdf[perf_start+ pd.Timedelta(1, "m"):perf_start + pd.Timedelta(4, "m")]
            cpu_list=list(mmmdf[aws_metric1])
            all_cpu_list.extend(cpu_list)
            average_cpu=np.mean(cpu_list)
            avg_perfomance_cpu_list.append(average_cpu)

            #print(list(mmmdf.cpuUtilization))
            if value.Labels[-1] == '1':
                first_perf_start=index
                #print(value.Labels)
        avg_cpu=np.mean(avg_perfomance_cpu_list)
        
        #avg_cpu_list.append(avg_cpu)

        # get number of errors from the start of first performance till the end of last scaling
        merr_df=err_df[first_perf_start:first_perf_start + pd.Timedelta(68, "m")] 
        err_list=list(merr_df[aws_metric0])
        total_err_list.append(sum(err_list))
        #print(total_err_list)



        ##================ get json K8s data ================
        #print(i)
        k8s_depl_path=i/"k8s-deploy-metrics.json"
        if platform=="k8s":
            k8s_df = pd.read_json(k8s_depl_path, lines=True)
            k8s_df['time'] = pd.to_datetime(k8s_df['time'],format='%d-%m-%Y %H:%M:%S') #26-03-2022 08:44:30
            k8s_df.set_index(['time'],inplace=True)
            k8s_df=k8s_df.tz_localize(tz='UTC') # localize, Pandas hates tz-naive timestamps

        ## ================ get json CloudWatch data ================
        aws_metric2="groupInServiceCapacity"
        csv_paths2 = (i/"csv").glob('*'+aws_metric2+'.csv')
        c_path2=list(csv_paths2)[0]
        cwdf2 = pd.read_csv(c_path2, parse_dates=['datetime'], index_col="datetime")


        ## ================ Process scaling data ================
        scaling_durations=[]
        #print(c_path2)
        for index,value in scaling_starts_df.iterrows():
            scale_start=index
            #print(scale_start)
            
            #print(ccwdf2)
            if platform=="k8s":
                kk8s_df=k8s_df[scale_start:scale_start+ pd.Timedelta(14, "m")]
                max_scaled=kk8s_df[kk8s_df['readyReplicas'].ge(6)]
            elif platform=="asg":
                ccwdf2=cwdf2[scale_start:scale_start+ pd.Timedelta(14, "m")] # only need to look 14 min ahead as it's the length of the scaling run
                max_scaled=ccwdf2[ccwdf2[aws_metric2].ge(3)]

            # print(ccwdf2.head(6))
            # print(max_scaled.head(2))

            if max_scaled.size > 0:
                cw_reach_max_time=max_scaled.index[0]
                duration = cw_reach_max_time - index
                duration_in_s = duration.total_seconds()
                #minutes = divmod(duration_in_s, 60)[0]
                #print(reach_max_time)
                #s = scaling_starts_df.index.get_loc(cw_reach_max_time, method='nearest')
                #max_minute=scaling_starts_df[reach_max_time]
                
                # print(index)
                # print(cw_reach_max_time)
                scaling_durations.append(duration_in_s)
            else:
                duration_in_s = 0
        avg_duration=np.mean(scaling_durations)
        all_duration_list.extend(scaling_durations)
        
        if np.isnan(avg_duration): avg_duration=0
        #avg_duration_list.append(avg_duration)

        print(f"[{platform}] [ CPU:{avg_cpu} | Dur:{avg_duration} | qps:{avg_qps} ]")

        ##================ generate rows ================
        # print(avg_duration_list)
        line = ""
        line = f"""<tr><td rowspan='1'>{count}: <a href='{i.name}'>{dir}</a></td><td></td><td></td>
                <td>Dura-tion s</td><td>qps</td><td>CPU %</td><td>Err</td>
                </tr><tr>"""
        ####====== Images
        for y in graphs:
            #print(i.name)
            img=f'{i.name}/csv/{y}'
            line+=f"<td><img alt='{y}' src='{img}' /></td>"

        ####====== Data

        #data=str(lst)[1:-1].replace(", ", "<br>")
        line+=f"<td>{list(map(int,scaling_durations))}<br><b>{avg_duration:.0f}</b></td>"
        line+=f"<td>{list(map(int,qps_lst))}<br><b>{avg_qps:.0f}</td>"
        line+=f"<td>{list(map(int,avg_perfomance_cpu_list))}<br><b>{avg_cpu:.0f}</b></td>"
        line+=f"<td><b>{total_err_list[-1]}</b></td>"

        line+="</tr>"
        #line+="</tr><tr><td rowspan='1'>~~~</td></tr>"
        iList.append(line)
    
    
    total=f"""<td>
        <h3>{tst} </h3>samples: <b>{count}</b><br>date: {nw}<br> </td><td></td><td></td>
        <td>max:<br>{max(all_duration_list):.0f}<br>min:<br>{min(all_duration_list):.0f}<br>mean:<br>{np.mean(all_duration_list):.0f}<br></td>
        <td>max:<br>{max(all_qps_list):.0f}<br>min:<br>{min(all_qps_list):.0f}<br>mean:<br>{np.mean(all_qps_list):.0f}<br></td>
        <td>max:<br>{max(all_cpu_list):.0f}<br>min:<br>{min(all_cpu_list):.0f}<br>mean:<br>{np.mean(all_cpu_list):.0f}<br></td>
        <td>max:<br>{max(total_err_list):.0f}<br>min:<br>{min(total_err_list):.0f}<br>mean:<br>{np.mean(total_err_list):.0f}<br></td>"""
    iList.append(total)
    iList.insert(0,total)
    imagesList = ''.join(iList)

    ##================ save HTML ================

    html_path=p/f'o_{tst}.html'
    header=f"<head><title>{tst} - Result Explorer</title>"
    header+="<style>table, th, td {border: 1px solid black;font-size: 20px;} body {font: 20px Arial, sans-serif;}</style></head>"


        
    # header+="</tr></table>"
    menuf=p/"o_menu.html"
    # the menu can be manually customized, will be generated if it does not exist
    if not menuf.exists():
    #================ generate menu
        
        menu=''
        def sorter(item):
            """sort by app and nodes"""
            x = ''.join(item.name.split("_")[2:4])
            return x

        menu+="<br><table><tr>"
        files = sorted(p.glob('*.html'), key=sorter)
        for h in files:
            # if h.name.find(f'{tst}')>0:
            #     test_name=f"<b>{h.stem}</b>"
            # else:
            #     test_name=f"{h.stem}"
            test_name=f"{h.stem}"
            if (files.index(h)% 2) == 0:
                menu+=f"\n<td><a href='{h.name}'>{test_name}</a>&nbsp;"
            else:
                menu+=f"<br><a href='{h.name}'>{test_name}</a>&nbsp;</td>"
        print("saving menu, exists:",menuf.exists() )
        with open(menuf, 'w') as mf:
            mf.write(menu)
    ## external menu:
    mf = open(menuf, "r")
    header+=mf.read()
    mf.close()
    
    header+="<table>"
    footer="</table>"

    page= header + imagesList
    a = HTML(page)
    html_src = a.data
    with open(html_path, 'w') as f:
        f.write(html_src)

    csv=""
    csv+=f'{tst},{count},{tst.split("_")[0]},{tst.split("_")[1]},'
    csv+=f'"{list(map(int,all_duration_list))}",{max(all_duration_list):.0f},{np.mean(all_duration_list):.0f},{min(all_duration_list):.0f},'
    csv+=f'"{list(map(int,all_qps_list))}",{max(all_qps_list):.0f},{np.mean(all_qps_list):.0f},{min(all_qps_list):.0f},'
    csv+=f'"{list(map(int,all_cpu_list))}",{max(all_cpu_list):.0f},{np.mean(all_cpu_list):.0f},{min(all_cpu_list):.0f},'
    csv+=f'"{total_err_list}",{max(total_err_list):.0f},{np.mean(total_err_list):.0f},{min(total_err_list):.0f}\n'

    csv_path=p/f'o_summary.csv'
    with open(csv_path, "a") as csvfile:
        csvfile.write(csv)

    #display(a, metadata=dict(isolated=True))


processing k8s_taewa_3lite_

c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.26_08-14_k8s_taewa_3lite_9d2d
[k8s] [ CPU:94.41083629469694 | Dur:423.961302 | qps:45953.34647384088 ]
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.27_09-43_k8s_taewa_3lite_0c47
[k8s] [ CPU:99.48881026760107 | Dur:352.6142853333333 | qps:53354.031314479034 ]
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.31_23-04_k8s_taewa_3lite_41cf
[k8s] [ CPU:99.37330319774668 | Dur:350.185189 | qps:53731.37938820106 ]
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.01_08-53_k8s_taewa_3lite_41cf
[k8s] [ CPU:99.37302925190663 | Dur:333.55981833333334 | qps:52817.18250391996 ]
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.01_21-08_k8s_taewa_3lite_41cf
[k8s] [ CPU:99.1839243936851 | Dur:384.0465236666667 | qps:53630.28242410013 ]
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.02_06-11_k8s_taewa_3lite_41cf
[k8s] [ CPU:96.51085788488048 | Dur:400.3023150000001 | qps:4